# Opportunity Owner Change Validation Process – Demo Notebook

This Jupyter notebook demonstrates an automated workflow to validate opportunity owner changes
using data structured like Dynamics CRM. The workflow compares Sales Rep intake form submissions 
(requesting owner changes) with current CRM data and previous audit outcomes, then flags any 
discrepancies and generates an approval recommendation.

- In a real-world scenario:
    * The Sales Rep submits a change request through an intake form (user_form_data.csv).
    * The CRM audit data and previous audit results would be retrieved via SQL queries 
      directly from the company database.
- In this demonstration:
    * All data is synthetic and loaded from CSV files to enable safe public sharing.

Follow each cell below for a well-documented, step-by-step validation process.

In [ ]:
# Import libraries
import pandas as pd
import os
os.makedirs('../output', exist_ok=True)

In [ ]:
# Load CSV files
df_form = pd.read_csv('../data/user_form_data.csv')
df_crm = pd.read_csv('../data/crm_audit_data.csv')
df_prev = pd.read_csv('../data/previous_audit_data.csv')

In [ ]:
# Add source info
df_form['Source'] = 'User Form'
df_crm['Source'] = 'CRM'
df_form['Opportunity URN'] = df_form['Opportunity URN'].astype(str)
df_crm['Opportunity URN'] = df_crm['Opportunity URN'].astype(str)
df_prev['Opportunity URN'] = df_prev['Opportunity URN'].astype(str)

In [ ]:
# Combine data
cols = ['Source','Opportunity URN','Original Close Date','Opportunity Name','Account Name',
        'Ult Parent Customer Urn','Ult Parent Name','Owner','Owner Employee ID','DC Owner',
        'DC Owner: Manager','Rep Type','Region','Annualized Amount','Type','Booking Event Type']
df_base = pd.concat([df_form[cols], df_crm[cols]], ignore_index=True)

In [ ]:
# Merge previous audit results
prev_map = df_prev.set_index('Opportunity URN')['Suggested Action'].to_dict()
df_base['Previous Audit Suggested Action'] = df_base['Opportunity URN'].map(prev_map).fillna('No previous audit')

In [ ]:
# Compare and flag changes
for col in ['Owner Change?','Region Change?','Rep Type Change?','QO Change?','QO Manager Change?']:
    df_base[col] = None

for urn in df_base['Opportunity URN'].unique():
    rows = df_base[df_base['Opportunity URN'] == urn]
    if len(rows) == 2:
        form = rows[rows['Source']=='User Form'].iloc[0]
        crm = rows[rows['Source']=='CRM'].iloc[0]
        for idx in rows.index:
            df_base.at[idx, 'Owner Change?'] = form['Owner'] != crm['Owner']
            df_base.at[idx, 'Region Change?'] = form['Region'] != crm['Region']
            df_base.at[idx, 'Rep Type Change?'] = form['Rep Type'] != crm['Rep Type']
            df_base.at[idx, 'QO Change?'] = form['Owner Employee ID'] != crm['Owner Employee ID']
            df_base.at[idx, 'QO Manager Change?'] = form['DC Owner: Manager'] != crm['DC Owner: Manager']

In [ ]:
# Generate status and guidance
def approval_logic(row):
    if row['Owner Change?']: return 'Review'
    elif row['Region Change?'] or row['Rep Type Change?']: return 'Adjust'
    elif row['QO Change?'] or row['QO Manager Change?']: return 'Adjust'
    return 'Approved'

def comms_guidance(row):
    if row['Owner Change?']: return 'Owner has changed – please review and validate.'
    if row['Region Change?']: return 'Region differs – check for update.'
    if row['Rep Type Change?']: return 'Rep Type mismatch – investigate.'
    if row['QO Change?']: return 'Quota Owner changed – validate.'
    if row['QO Manager Change?']: return 'Quota Owner Manager changed – review.'
    return 'No discrepancies detected.'

df_base['Approval Status'] = df_base.apply(approval_logic, axis=1)
df_base['Approval Comms Guidance'] = df_base.apply(comms_guidance, axis=1)

In [ ]:
# Save output
df_base.to_csv('../output/final_result.csv', index=False)
print('✅ Output saved to ../output/final_result.csv')